In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

In [2]:
def getHistoricalDataOffense():
    '''
    Scrapes nfl.com for skill player stats from 2010 to 2017.
    Places them in a dictionary format so that each position can be transformed into a dataframe
    '''
    playerData = {}
    playerCol = {}
    for pos in ['QUARTERBACK','RUNNING_BACK','WIDE_RECEIVER','TIGHT_END']:
        playerData[pos]=[]
        for year in range(2010,2018):
            time.sleep(1)
            table= []
            link = f"http://www.nfl.com/stats/categorystats?archive=false&conference=null&statisticPositionCategory={pos}&season={year}&seasonType=REG&experience=&tabSeq=1&qualified=false&Submit=Go"
            time.sleep(1)
            response = requests.get(link)
            html = response.text.encode()
            soup = BeautifulSoup(html,'html.parser')
            data=[x for x in soup.find('table',{'id':'result'}).text.replace('\t','').split('\n') if x!='']
            split=data.index('1')
            if year==2010:
                playerCol[pos]=data[:split]+['YEAR']
            data=data[split:]
            while len(data)>0:
                table.append(data[:split]+[year])
                data=data[split:]
            playerData[pos].append(table)
    return playerData, playerCol

def getHistoricalDataKicker():
    '''
    Scrapes nfl.com for kickers stats from 2010 to 2017.
    Places them in a dictionary format so that each position can be transformed into a dataframe
    '''
    kickerData = []
    for year in range(2010,2018):
        time.sleep(1)
        table = []
        link = f"http://www.nfl.com/stats/categorystats?archive=false&conference=null&statisticPositionCategory=FIELD_GOAL_KICKER&season={year}&seasonType=REG&experience=&tabSeq=1&qualified=false&Submit=Go"
        time.sleep(1)
        response = requests.get(link)
        html = response.text.encode()
        soup = BeautifulSoup(html,'html.parser')   
        data=[x for x in soup.find('table',{'id':'result'}).text.replace('\t','').split('\n') if x!='']
        pref = data[:7]
        data=data[7:]
        split=data.index('1')
        if year==2010:
            kickerCol = data[:split]+['YEAR']
        data=data[split:]
        while len(data)>0:
            table.append(data[:split]+[year])
            data=data[split:]
        kickerData.append(table)
    return kickerData, kickerCol, pref
offenseData, offenseCol = getHistoricalDataOffense()
kickerData, kickerCol, kickerPref = getHistoricalDataKicker()


In [3]:
def createPosDataFrame(data,col):
    if type(data)==dict:
        return {pos:pd.DataFrame(columns=col[pos],data=[player for year in data[pos] for player in year]) for pos in data}
    else:
        return pd.DataFrame(columns=col,data=[player for year in data for player in year])

In [4]:
kickerCol2=[]
kickerPref2 = kickerPref[:][1:-1]
for col in kickerCol:
    if col=='A-M':
        kickerCol2.append(kickerPref2[0]+'_'+col)
    elif col=='Pct' and colPref!='FG Att' and colPref!='XP Att':
        kickerCol2.append(kickerPref2[0]+'_'+col)
        kickerPref2=kickerPref2[1:]
    else:
        kickerCol2.append(col)
    colPref=col
for i in range(len(kickerCol2)):
    if kickerCol2[i]=='Pct' and 'Pct' in kickerCol2[i+1:]:
        kickerCol2[i]='FG_Pct'
    elif kickerCol2[i]=='Pct':
        kickerCol2[i]='XP_Pct'
kickerCol2

['Rk',
 'Player',
 'Team',
 'Pos',
 'FGM',
 'FG Att',
 'FG_Pct',
 'Blk',
 'Lng',
 '1-19_A-M',
 '1-19_Pct',
 '20-29_A-M',
 '20-29_Pct',
 '30-39_A-M',
 '30-39_Pct',
 '40-49_A-M',
 '40-49_Pct',
 '50+_A-M',
 '50+_Pct',
 'XPM',
 'XP Att',
 'XP_Pct',
 'Blk',
 'YEAR']

In [5]:
offenseDataFrames = createPosDataFrame(offenseData,offenseCol)
kickerDataFrame = createPosDataFrame(kickerData,kickerCol2)
kickerDataFrame

,Rk,Player,Team,Pos,FGM,FG Att,FG_Pct,Blk,Lng,1-19_A-M,...,30-39_Pct,40-49_A-M,40-49_Pct,50+_A-M,50+_Pct,XPM,XP Att,XP_Pct,Blk,YEAR
0,1,Josh Brown,STL,K,33,39,85,2,53,0-0,...,86,9-7,78,4-3,75,26,27,96,0,2010.0
1,1,Sebastian Janikowski,OAK,K,33,41,80,0,59,0-0,...,93,12-8,67,7-4,57,43,43,100,0,2010.0
2,3,David Akers,PHI,K,32,38,84,1,50,0-0,...,83,11-9,82,3-1,33,47,47,100,0,2010.0
3,4,Dan Carpenter,MIA,K,30,41,73,2,60,1-1,...,100,18-11,61,8-4,50,25,25,100,0,2010.0
4,4,Nick Folk,NYJ,K,30,39,77,1,56,1-1,...,88,6-3,50,5-2,40,37,37,100,0,2010.0
5,6,Matt Bryant,ATL,K,28,31,90,1,51,0-0,...,89,10-8,80,1-1,100,44,44,100,0,2010.0
6,7,Neil Rackers,HOU,K,27,30,90,0,57,0-0,...,100,7-5,71,4-3,75,43,43,100,0,2010.0
7,8,Billy Cundiff,BAL,K,26,29,90,0,49,0-0,...,89,8-7,88,1-0,0,39,39,100,0,2010.0
8,8,Adam Vinatieri,IND,K,26,28,93,1,48,0-0,...,90,10-9,90,0-0,0,51,51,100,0,2010.0
9,10,Robbie Gould,CHI,K,25,30,83,1,54,1-1,...,89,7-4,57,4-3,75,35,35,100,0,2010.0


In [6]:
for key in offenseDataFrames:
    offenseDataFrames[key].to_csv(f"{key}.csv",index=False)
kickerDataFrame.to_csv('kicker.csv',index=False)